## Plotting

This notebook creates visuals of the runoff data for all 3 Models. The data has been loaded elsewhere and converted to a locally saved CSV to the address memory capacity issues encountered when processing large RGI regions. 

Last edited: July 13, 2023 | FFW

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date
import collections
import datetime
import os
import xarray as xr
from cycler import cycler
import matplotlib.patches as mpatches

In [ ]:
glacier_models = ['GloGEM', 'OGGM', 'PyGEM']

modelnames_all = ['BCC-CSM2-MR', 'CESM2', 'CESM2-WACCM', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 
                  'INM-CM4-8', 'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM']

scenarios = ['ssp126','ssp245','ssp370','ssp585']

Alpine_basins = {'FRASER':'4410', 'COLUMBIA':'4406', 'NELSON':'4125', 'MACKENZIE':'4123'}

basins = ['FRASER', 'COLUMBIA', 'NELSON', 'MACKENZIE']

basinstext = ['Fraser', 'Columbia', 'Nelson', 'Mackenzie']

fpath0 = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Load Separate/RGI 02/'

#### Importing CSV files

In [ ]:
# Create new index using pandas date_range function
start_date = datetime.date(2000, 1, 1)
end_date = datetime.date(2100, 12, 1)
new_indices = pd.date_range(start_date, end_date, freq='MS').strftime('%Y-%m').tolist()

In [ ]:
#Loading data, applying datetime indices, and creating annual sum dfs
all_rf_data = {}
all_rf_data_annual = {}
for g, gmodel in enumerate(glacier_models):
    all_rf_data[gmodel] = {}
    all_rf_data_annual[gmodel] = {}
    fpath = fpath0 + '/' + gmodel + '/'
    for s, SSP in enumerate(scenarios):
        all_rf_data[gmodel][SSP] = {}
        all_rf_data_annual[gmodel][SSP] = {}
        for b, basin in enumerate(basins):
            all_rf_data[gmodel][SSP][basin] = {}
            all_rf_data_annual[gmodel][SSP][basin] = {}
            for m, GCM in enumerate(modelnames_all):
                fname = f"runoff_{GCM}_{SSP}_{basin}.csv"
                temp_df = pd.read_csv(fpath + fname, index_col = 0)
                all_rf_data[gmodel][SSP][basin][GCM] = temp_df
                all_rf_data[gmodel][SSP][basin][GCM].index = new_indices
                all_rf_data[gmodel][SSP][basin][GCM].index = pd.to_datetime(new_indices)
                all_rf_data_annual[gmodel][SSP][basin][GCM] = all_rf_data[gmodel][SSP][basin][GCM].resample('A').sum()

In [ ]:
# Taking multi-GCM means and quartiles
GCM_mean = {}
GCM_q1 = {}
GCM_q3 = {}
for g, gmodel in enumerate(glacier_models):
    GCM_mean[gmodel] = {}
    GCM_q1[gmodel] = {}
    GCM_q3[gmodel] = {}
    for s, SSP in enumerate(scenarios):
        GCM_mean[gmodel][SSP] = {}
        GCM_q1[gmodel][SSP] = {}
        GCM_q3[gmodel][SSP] = {}
        for b, basin in enumerate(basins):
            df_dict = all_rf_data_annual[gmodel][SSP][basin]
           
            df_mean = pd.concat(df_dict.values(), axis=1).mean(axis=1)
            GCM_mean[gmodel][SSP][basin] = df_mean

            df_q1 = pd.concat(df_dict.values(), axis=1).quantile(q=0.25, axis=1)
            GCM_q1[gmodel][SSP][basin] = df_q1

            df_q3 = pd.concat(df_dict.values(), axis=1).quantile(q=0.75, axis=1)
            GCM_q3[gmodel][SSP][basin] = df_q3

In [ ]:
#Creating time values and color schemes
yrs = np.arange(2000, 2101)
yrs_dt = pd.to_datetime([str(y) for y in yrs])

colorschemes = {}

colors_glo =  plt.colormaps['Greens']
line_colors_glo = colors_glo(np.linspace(0.2, 0.6, num = 12))
glo_cycler = cycler(color = line_colors_glo)
colorschemes['GloGEM'] = glo_cycler

colors_OG =  plt.colormaps['Blues']
line_colors_OG = colors_OG(np.linspace(0.2, 0.6,num = 12))
OG_cycler = cycler(color = line_colors_OG)
colorschemes['OGGM'] = OG_cycler

colors_py =  plt.colormaps['Purples']
line_colors_py = colors_py(np.linspace(0.2, 0.6,num = 12))
py_cycler = cycler(color = line_colors_py)
colorschemes['PyGEM'] = py_cycler

colors = ['darkgreen', 'royalblue', 'purple']
fill_colors = ['green', 'dodgerblue', 'purple']

In [ ]:
fig, axs = plt.subplots(len(basins), len(scenarios), figsize=(10, 2.4*len(basins)), sharex =True)

for g, gmodel in enumerate(glacier_models):
    for m, GCM in enumerate(modelnames_all):
        for s, SSP in enumerate(scenarios):
            for b, basin in enumerate(basins):
                if gmodel == 'OGGM':
                    axs[b,s].plot(yrs_dt[0:-1], all_rf_data_annual[gmodel][SSP][basin][GCM][0:-1], color=axs[b, s].set_prop_cycle(colorschemes[gmodel]), alpha = 0.25)
                    axs[b,s].plot(yrs_dt[0:-1], GCM_mean[gmodel][SSP][basin][0:-1], color = colors[g], linewidth = 0.9)
                    axs[b,s].plot(yrs_dt[0:-1], GCM_q1[gmodel][SSP][basin][0:-1], color = colors[g], linewidth = 0.4)
                    axs[b,s].plot(yrs_dt[0:-1], GCM_q3[gmodel][SSP][basin][0:-1], color = colors[g], linewidth = 0.4)
                    axs[b,s].fill_between(yrs_dt[0:-1], GCM_q1[gmodel][SSP][basin][0:-1], GCM_q3[gmodel][SSP][basin][0:-1], color = fill_colors[g])
                else:
                    axs[b,s].plot(yrs_dt, all_rf_data_annual[gmodel][SSP][basin][GCM], color=axs[b, s].set_prop_cycle(colorschemes[gmodel]), alpha = 0.25)
                    axs[b,s].plot(yrs_dt, GCM_mean[gmodel][SSP][basin], color = colors[g], linewidth = 0.9)
                    axs[b,s].plot(yrs_dt, GCM_q1[gmodel][SSP][basin], color = colors[g], linewidth = 0.4)
                    axs[b,s].plot(yrs_dt, GCM_q3[gmodel][SSP][basin], color = colors[g], linewidth = 0.4)
                    axs[b,s].fill_between(yrs_dt, GCM_q1[gmodel][SSP][basin], GCM_q3[gmodel][SSP][basin], color = fill_colors[g])
                axs[b,s].set(xlim=(pd.to_datetime('2000-01-01'), pd.to_datetime('2100-01-01')))

         #Setting x and y labels and making y limits uniform within basins
                if b == (len(basins)-1):
                    for sub_s in range(4):  # Use a different variable name for the inner loop
                        axs[b, sub_s].set_xlabel('Year')
                        axs[b, sub_s].set_xticks([pd.to_datetime('2025'),pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')], [2025, 2050, 2075, 2100])
                else:
                    axs[b, s].set_xlabel(None) 
                
                if s == 0:                                                                    #Setting basin labels
                    for sub_b in range(len(basins)):
                        axs[sub_b,s].set_ylabel(basinstext[sub_b]+ r' $[km^3]$')
                if s != 0:
                    axs[b, s].set_ylabel(None)
                    axs[b, s].set_yticklabels('')

green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
purple_patch = mpatches.Patch(color='purple', label='PyGEM') 
blue_patch = mpatches.Patch(color='royalblue', label='OGGM')
axs[0,0].legend(handles=[green_patch, purple_patch, blue_patch], bbox_to_anchor=(3.15, 1.5), ncol=3)

plt.suptitle('GloGEM, PyGEM, and OGGM Runoff Projections for Major Western Canadian River Basins', x=0.48, y=0.98)
plt.title('SSP 126                            SSP 245                           SSP 370                            SSP 585', x=-1.3, y=(1.176* len(basins))) 

### Reading Out Monthly Runoff in a Single CSV

In [ ]:
#Using the GloGEM datetime index
indices = all_rf_data[gmodel][SSP][basin][GCM].index

#Creating dataframes of SSP, basin, and GCM containing all 3 global glacier models
out_df = {}
for s, SSP in enumerate(scenarios):
    out_df[SSP] = {}
    for b, basin in enumerate(basins):
        out_df[SSP][basin] = {}
        for m, model in enumerate(modelnames_all):
            glo_values = all_rf_data['GloGEM'][SSP][basin][model].values.flatten()
            pygem_values = all_rf_data['PyGEM'][SSP][basin][model].values.flatten()
            oggm_values = all_rf_data['OGGM'][SSP][basin][model].values.flatten()

            out_df[SSP][basin][model] = pd.DataFrame(
                {
                    'GloGEM': glo_values,
                    'PyGEM': pygem_values,
                    'OGGM': oggm_values,
                },
                index=indices
            )

In [ ]:
# Define the directory to save the CSV files
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Runoff/RGI 01/'

for SSP in out_df:
    for basin in out_df[SSP]:
        for GCM in out_df[SSP][basin]:
            fname = f"runoff_{GCM}_{SSP}_{basin}.csv"

            # Define the full path of the output file
            output_path = os.path.join(output_dir, fname)

            # Save the DataFrame as CSV
            out_df[SSP][basin][GCM].to_csv(output_path, header=True, index=True)